Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `???` or "YOUR ANSWER HERE".

---

# CSCI 3155 Assignment 4 : Semantics and Map/Reduce/Filter.

This assignment asks you to write scala programs. 

**Restrictions** apply to each problem in terms of forbidden Scala features and API functions. Please read them carefully and ask for clarifications from the course staff over Piazza or during office hours if unsure.

Note: `???` indicates that there is a missing function or code fragment that needs to be filled in. In scala, 
it is also a macro that throws a `NotImplemented` exception. Make sure that you remove the `???` and replace it with the answer. 

Use the test cases provided to test them. You are also encouraged to write your own test cases to help debug your work. However, please delete any extra cells you may have created lest they break our autograder.

**Very Important:** Please run the cell that defines the functions `passed` and `testWithMessage` below whenever you restart the notebook.

### Your Name Here

In [1]:
// TEST HELPER

// FIRST RUN THIS CELL EVERY TIME YOU START THE NOTEBOOK
def passed(points: Int) {
    require(points >=0)
    if (points == 1) print(s"\n*** Tests Passed (1 point) ***\n")
    else print(s"\n*** Tests Passed ($points points) ***\n")
}

def testWithMessage[T](v1: T, expected: T, testID: String) = { 
    println(s"Test $testID"); 
    println(s"\t Your code returned: $v1, Expected: $expected")
    assert (v1 == expected, s"Test $testID FAILED.")
    println("\t Passed!")
}
/*

def testWithMessage(v1: Double, expected: Double, testID: String) = {
    val tolerance = 1E-5
    println(s"Test $testID -- comparing with tolerance $tolerance."); 
    println(s"\t Expected: $expected, your code returned: $v1")
    assert (math.abs(v1-expected) <= tolerance, s"Test $testID FAILED.")
    println("\t Passed!")
}
*/

defined function passed
defined function testWithMessage

## Problem 1 (45 points)

In this assignment, we will implement regular expression 
matching using semantic rules. H/T to Prof. Matt Might : https://matt.might.net/articles/implementation-of-regular-expression-matching-in-scheme-with-derivatives/ 



$$\newcommand\Regex{\mathbf{Regex}}
\newcommand\trm[1]{\mathit{#1}}$$

The grammar of regular expressions is given by 
$$\begin{array}{rll}
\Regex & \Rightarrow \trm{Atom}(\mathbf{String}) & \text{match the string}\\ 
& | \ \trm{EmptyStr} & \text{match empty string} \\ 
& | \ \trm{Null} & \text{a regular expression that does not match any string} \\ 
& |\ \trm{Seq}(\Regex, \Regex) & \text{match the first regex and then the second}\\ 
& |\ \trm{Or}(\Regex, \Regex) & \text{Or of two regular expressions}\\ 
& |\ \trm{Star} (\Regex) & \text{Kleene-star: match zero or more occurrences of regex}\\ 
\end{array} $$ 

A regex such as `("hello")* | ( "world" ; "csci3155) *` is expressed in our abstract syntax by the term
~~~
Or( 
    Star(Atom("hello")), 
    Star( Seq( Atom("world"), Atom("csci3155")) 
  )
~~~

Note that `Null` is not something that a user writes or for that matter `EmptyStr`. But we will need them 
for our match: these will typically not be part of the "concrete" syntax that a user sees.

### Regular Expression Matching

Given a Regex `("hello")* | ( "world" ; "csci3155) *` 
 - The string `"worldcsci3155worldcsci3155"` matches the regex. 
 - Likewise the string `"hellohellohello"` also matches
the regex. 
 - However, the string `"helloworld"` does not match the regex.

We will use semantic rules to write a matcher function `match(str, regex)` that returns `true` if a string matches a regular expression and `false` otherwise. 

To do so, we proceed in three stages: 
  - We will define the `accepts` function.
  - We will define the `simplifyNull` function to remove `null` wherever possible.
  - We will define the `derivative` of a regex with respect to a character.
  - We will finally put the two together to define a `match` function. 
  
Strap on your seatbelts and join us for the regex match!!




### Accepts Function

The accepts function specified below using semantic rules encodes whether a regular expression accepts the _empty string_.


$$\newcommand\semRule[3]{\begin{array}{c} #1 \\ \hline #2 \\ \end{array}\ (\text{#3}) }\newcommand\acc{\mathsf{accepts}} \newcommand\true{\mathit{true}} \newcommand\false{\mathit{false}}\newcommand\sNull{\mathsf{simplifyNull}}$$

Here are three simple rules. They state that empty string regex accepts the empty string, $\trm{Null}$ rejects the empty string and a regex of the form $\trm{Star}(t)$ accepts the empty string as long as $t \not= Null$. Additionally, there are two rules for $\trm{Atom}$, as well.

$$\semRule{}{\acc(\trm{EmptyStr}) = \true}{empty-str}$$
$$\semRule{}{\acc(\trm{Null}) = \false}{null}$$
$$\semRule{s \not= ""}{\acc(\trm{Atom}(s)) = \false}{atom-non-empty}$$
$$\semRule{s = ""}{\acc(\trm{Atom}(s)) = \true}{atom-empty}$$
$$\semRule{t \not= Null}{\acc(\trm{Star}(t)) = \true}{kleene-star}$$

The rules for Or and Seq.
$$\semRule{ \acc(t_1) \ \textbf{or}\ \acc(t_2) }{\acc(Or(t_1, t_2)) = \true }{or-rule}$$
$$\semRule{ \acc(t_1) \ \textbf{and}\ \acc(t_2) }{\acc(Seq(t_1, t_2)) = \true }{or-rule}$$

We will skip the remaining rules for $\trm{Or}(...)$, $\trm{Seq}(...)$ and $\trm{Star}(\trm{Null})$ for which the function evaluates to  $\false$ but you should be able to figure those out yourself.

Run the definitions for Regex below and implement the function `accepts(r: Regex): Boolean` as specified by the rules above.


In [2]:
import scala.language.postfixOps
/* Please ensure that you run this cell */
sealed trait Regex { // Let's overload some operators for testing purposes.
    def | (r2: Regex) = Or(this, r2) // or
    def o (r2: Regex) = Seq(this, r2) // semicolon operator
    def star = Star(this) // Kleene star
}
case object EmptyStr extends Regex
case object Null extends Regex
case class Atom(s: String) extends Regex
case class Or(r1: Regex, r2: Regex) extends Regex
case class Seq(r1: Regex, r2: Regex) extends Regex
case class Star(r: Regex) extends Regex

/*-- implicitly convert strings to Atoms --*/
implicit def to_regex(s:String): Regex = Atom(s)



import scala.language.postfixOps
/* Please ensure that you run this cell */

defined trait Regex
defined object EmptyStr
defined object Null
defined class Atom
defined class Or
defined class Seq
defined class Star
defined function to_regex

In [3]:
val r1 = ("x" o ("_yellow").star).star | ("_white")

r1: Or = Or(
  r1 = Star(r = Seq(r1 = Atom(s = "x"), r2 = Star(r = Atom(s = "_yellow")))),
  r2 = Atom(s = "_white")
)

In [4]:
/* Implement the accepts function */
// YOUR CODE HERE
def accepts(r: Regex): Boolean=
{
    r match
    {
        case EmptyStr => true;
        case Null => false;
        case Atom(s) => if(s == ""){true} else{false}
        case Star(e1) => if(e1 == Null){false} else{true}
        case Or(e1, e2) => accepts(e1) || accepts(e2)
        case Seq(e1, e2) =>accepts(e1) && accepts(e2)
        
        
    }
}

defined function accepts

In [5]:
val r1: Regex = "" // This is where the implicit function we defined above helps us
assert(accepts(r1), "Test # 1 failed")

val r2 = "".star
assert(accepts(r2), "Test # 2 failed")

val r3 = "hello" | "world"
assert(!accepts(r3), "Test # 3 failed")

val r4 = "hello" | "world".star 
assert (accepts(r4), "Test # 4 failed")

val r5: Regex = "hello"
assert(!accepts(r5), "Test # 5 failed")

val r6: Regex = "hello" o "world".star
assert(!accepts(r6), "Test #6 failed")

val r7: Regex = "" o "world"
assert(!accepts(r7), "Test # 7 failed")

val r8: Regex = ("" o "world".star ) |  "csci3155"o "hello".star
assert(accepts(r8), "Test # 8 failed")

val r9: Regex = ("" o "world".star ) |  ("csci3155"o "hello".star)
assert(accepts(r9), "Test # 9 failed")

val r10: Regex = Null | "hello"
assert(!accepts(r10), "Test # 10 failed")

val r11:Regex =  Null.star
assert(!accepts(r11), "Test #11 failed")

passed(10)


*** Tests Passed (10 points) ***


r1: Regex = Atom(s = "")
r2: Star = Star(r = Atom(s = ""))
r3: Or = Or(r1 = Atom(s = "hello"), r2 = Atom(s = "world"))
r4: Or = Or(r1 = Atom(s = "hello"), r2 = Star(r = Atom(s = "world")))
r5: Regex = Atom(s = "hello")
r6: Regex = Seq(r1 = Atom(s = "hello"), r2 = Star(r = Atom(s = "world")))
r7: Regex = Seq(r1 = Atom(s = ""), r2 = Atom(s = "world"))
r8: Regex = Seq(
  r1 = Or(
    r1 = Seq(r1 = Atom(s = ""), r2 = Star(r = Atom(s = "world"))),
    r2 = Atom(s = "csci3155")
  ),
  r2 = Star(r = Atom(s = "hello"))
)
r9: Regex = Or(
  r1 = Seq(r1 = Atom(s = ""), r2 = Star(r = Atom(s = "world"))),
  r2 = Seq(r1 = Atom(s = "csci3155"), r2 = Star(r = Atom(s = "hello")))
)
r10: Regex = Or(r1 = Null, r2 = Atom(s = "hello"))
r11: Regex = Star(r = Null)

## Simplify Null 

Next we specify some rules for getting rid of $\trm{Null}$. These rules are deliberately setup to avoid doing expensive "full" simplifications. Please read them carefully and implement  them as specified below.

If a term is an atom or null or an empty string, simplifying the term just yields the original term  back.
$$\semRule{t \in \{  \trm{Atom}(s), \trm{Null}, \trm{EmptyStr} \} }{\sNull(t) = t }{simplify-base}$$

Here are the rules for $\trm{Or}$: if one of the two arguments simplifies to $\trm{Null}$, we remove $\trm{Or}$ and replace it by the simplified version of the other term.

$$\semRule{\sNull(t_1) = \trm{Null} }{\sNull( \trm{Or}(t_1, t_2) ) = \sNull(t_2) }{simplify-or-1}$$

$$\semRule{\sNull(t_1) = s_1, s_1 \not= \trm{Null}, \text{and}\ \sNull(t_2) = \trm{Null} }{\sNull( \trm{Or}(t_1, t_2) ) = s_1 }{simplify-or-2}$$

$$\semRule{\sNull(t_1) = s_1,\ s_1 \not= \trm{Null},\ \sNull(t_2) = s_2,\ \text{and}\ s_2 \not= \trm{Null} }{\sNull( \trm{Or}(t_1, t_2) ) = \trm{Or}(s_1, s_2) }{simplify-or-2}$$

Here are the rules for $\trm{Seq}$:


$$\semRule{\sNull(t_1) = \trm{Null} }{\sNull( \trm{Seq}(t_1, t_2) ) = \trm{Null} }{simplify-seq-1}$$

$$\semRule{\sNull(t_1)= s_1\ \text{and}\ s_1 \not= \trm{Null}}{\sNull( \trm{Seq}(t_1, t_2) ) = \trm{Seq}(s_1, t_2) }{simplify-seq-2}$$

Notice that we do not bother simplifying $t_2$ for the $\trm{Seq}$ operator. We simplify $t_1$ and the two rules do different things based on whether $t_1$ simplifies to $\trm{Null}$ or not.

Here are the rules for $\trm{Star}$:
$$\semRule{\sNull(t_1) = \trm{Null} }{\sNull( \trm{Star}(t_1) ) = \trm{Null} }{simplify-star-1}$$
$$\semRule{\sNull(t_1)= s_1\ \text{and}\ s_1 \not= \trm{Null}}{\sNull( \trm{Star}(t_1) ) = \trm{Star}(s_1) }{simplify-star-2}$$

Implement a function `simplifyNull(r: Regex): Regex` following the specifications above.

In [6]:
// YOUR CODE HERE
def simplifyNull(r: Regex): Regex = 
{
    r match
    {
        case EmptyStr => r
        case Null => r
        case Atom(s) => r
        case Star(e1) => if (simplifyNull(e1) == Null) {Null} else{Star(simplifyNull(e1))}
        case Or(t1, t2) => 
        {
            if(simplifyNull(t1) == Null) 
                simplifyNull(t2)
            
            else if(simplifyNull(t2) == Null) 
                    simplifyNull(t1) 
            
            else
                Or(simplifyNull(t1),simplifyNull(t2))
        }
        case Seq(e1, e2) => 
        {
            if(simplifyNull(e1) == Null)
                Null
            else
                Seq(simplifyNull(e1), e2)
                
        }
    }
}

defined function simplifyNull

In [7]:
val r1 = Atom("hello") | (Null o Atom("World"))
val e1 = Atom("hello")
testWithMessage(simplifyNull(r1), e1, "1")

val r2 = Atom("hello") | (Null.star o Atom("World"))
val e2 = Atom("hello")
testWithMessage(simplifyNull(r2), e2, "2")


val r3 = (Null.star o Atom("World")) | Atom("excellent")
val e3 = Atom("excellent")
testWithMessage(simplifyNull(r3), e3, "3")

val r4 = (Null.star o Atom("sophisticated")) | (Null.star o Null)
val e4 = Null
testWithMessage(simplifyNull(r4), e4, "4")

val r5 = Atom("star rise") | r4.star | Atom("very nice")
val e5 = Atom("star rise") | Atom("very nice")
testWithMessage(simplifyNull(r5), e5, "5")
passed(10)

Test 1
	 Your code returned: Atom(hello), Expected: Atom(hello)
	 Passed!
Test 2
	 Your code returned: Atom(hello), Expected: Atom(hello)
	 Passed!
Test 3
	 Your code returned: Atom(excellent), Expected: Atom(excellent)
	 Passed!
Test 4
	 Your code returned: Null, Expected: Null
	 Passed!
Test 5
	 Your code returned: Or(Atom(star rise),Atom(very nice)), Expected: Or(Atom(star rise),Atom(very nice))
	 Passed!

*** Tests Passed (10 points) ***


r1: Or = Or(r1 = Atom(s = "hello"), r2 = Seq(r1 = Null, r2 = Atom(s = "World")))
e1: Atom = Atom(s = "hello")
r2: Or = Or(
  r1 = Atom(s = "hello"),
  r2 = Seq(r1 = Star(r = Null), r2 = Atom(s = "World"))
)
e2: Atom = Atom(s = "hello")
r3: Or = Or(
  r1 = Seq(r1 = Star(r = Null), r2 = Atom(s = "World")),
  r2 = Atom(s = "excellent")
)
e3: Atom = Atom(s = "excellent")
r4: Or = Or(
  r1 = Seq(r1 = Star(r = Null), r2 = Atom(s = "sophisticated")),
  r2 = Seq(r1 = Star(r = Null), r2 = Null)
)
e4: Null.type = Null
r5: Or = Or(
  r1 = Or(
    r1 = Atom(s = "star rise"),
    r2 = Star(
      r = Or(
        r1 = Seq(r1 = Star(r = Null), r2 = Atom(s = "sophisticated")),
        r2 = Seq(r1 = Star(r = Null), r2 = Null)
      )
    )
  ),
  r2 = Atom(s = "very nice")
)
e5: Or = Or(r1 = Atom(s = "star rise"), r2 = Atom(s = "very nice"))

In [8]:
//Tests for Seq
val r6 = Null o "Python"
val e6 = Null
testWithMessage(simplifyNull(r6), e6, "6")

println("Remember according to semantic rules for seq, you should not simplify the second argument")
val r7 =  "Python" o Null.star
val e7 = r7
testWithMessage(simplifyNull(r7), e7, "7")

val r8 =  ("Python" | Null.star) o ( Null.star o "Scala")
val e8 = "Python" o (Null.star o "Scala")
testWithMessage(simplifyNull(r8), e8, "8")

passed(5)

Test 6
	 Your code returned: Null, Expected: Null
	 Passed!
Remember according to semantic rules for seq, you should not simplify the second argument
Test 7
	 Your code returned: Seq(Atom(Python),Star(Null)), Expected: Seq(Atom(Python),Star(Null))
	 Passed!
Test 8
	 Your code returned: Seq(Atom(Python),Seq(Star(Null),Atom(Scala))), Expected: Seq(Atom(Python),Seq(Star(Null),Atom(Scala)))
	 Passed!

*** Tests Passed (5 points) ***


r6: Seq = Seq(r1 = Null, r2 = Atom(s = "Python"))
e6: Null.type = Null
r7: Seq = Seq(r1 = Atom(s = "Python"), r2 = Star(r = Null))
e7: Seq = Seq(r1 = Atom(s = "Python"), r2 = Star(r = Null))
r8: Seq = Seq(
  r1 = Or(r1 = Atom(s = "Python"), r2 = Star(r = Null)),
  r2 = Seq(r1 = Star(r = Null), r2 = Atom(s = "Scala"))
)
e8: Seq = Seq(
  r1 = Atom(s = "Python"),
  r2 = Seq(r1 = Star(r = Null), r2 = Atom(s = "Scala"))
)

## Derivatives 

We will now define the "derivative" of a regular expression `r` with respect to a characted `c` $\delta(r, c)$ using the semantic rules below.

$$\semRule{r \in \{  \trm{EmptyStr}, \trm{Null} \} }{\delta(r, c) = \trm{Null}}{deriv-empty-null}$$

Rules for `Atom`. For a string $s$, we will denote its first characted by $s.head$ and the substring from second position to end as $s.tail$.

$$\semRule{}{\delta(\trm{Atom}(""), c) = \trm{Null}}{deriv-empty-atom}$$
$$\semRule{s.head = c}{\delta(\trm{Atom}(s), c) = \trm{Atom}(s.tail)}{deriv-atom-1}$$
$$\semRule{s.head \not= c}{\delta(\trm{Atom}(s), c) = \trm{Null} }{deriv-atom-2}$$

Notice that if an atom starts with the character `c`, then the derivative is the tail of the contained string. But if an atom does not start with the character `c`, its derivative is simply null denoting a match failure.

Rules for `Seq`.

If the first sub-term $t_1$ is not accepting then we take the derivative of $t_1$ as specified below:

$$\semRule{\acc(t_1) = \false,\ \text{and}\  s_1 = \delta(t_1, c) }{\delta(\trm{Seq}(t_1, t_2), c)) = \trm{Seq}(s_1, t_2)}{deriv-seq-1}$$

If the first sub-term $t_1$ is accepting, then the derivative of $\trm{Seq}(t_1, t_2)$ is given by the following expression obtained by taking derivative of both subterms. 

$$\semRule{\acc(t_1) = \true,\ s_1 = \delta(t_1, c),\ \text{and}\ s_2 = \delta(t_2, c) }{\delta(\trm{Seq}(t_1, t_2), c)) = \trm{Or}( \trm{Seq}(s_1, t_2), s_2 )}{deriv-seq-2}$$

Rules of `Or`.
$$\semRule{}{\delta(\trm{Or}(t_1, t_2), c)) = \trm{Or}(\delta(t_1, c), \delta(t_2, c))}{deriv-or-1}$$

Rule for `Star`.

$$\semRule{}{\delta(\trm{Star}(t)), c) = \trm{Seq}(\delta(t, c), \trm{Star}(t))}{deriv-star-1}$$

Implement the function `delta(r: Regex, c: Char): Regex` using the semantic rules above.

In [9]:
// YOUR CODE HERE
def delta(r: Regex, c: Char): Regex = 
{
    r match
    {
        case EmptyStr => Null
        case Null => r
        case Atom(s) => if(s == "") {Null} else if(s.head != c) {Null} else{s.tail}
        case Star(e1) => Seq(delta(e1, c), Star(e1))
        case Or(e1, e2) => Or(delta(e1, c), delta(e2,c))
        case Seq(e1, e2) =>
        {
            if(accepts(e1) == false)
                Seq(delta(e1, c), e2)
            else
            {
                val s1 = delta(e1, c);
                val s2 = delta(e2, c);
                Or(Seq(s1, e2), s2)
            }
        }
    }
}

defined function delta

In [10]:
// Test for Atom
val r1 = Atom("hello")
val e1 = Atom("ello")
testWithMessage(delta(r1, 'h'), e1, "#1")

val r2 = Atom("hello")
val e2 = Null
testWithMessage(delta(r2, 'c'), e2, "#2")

// Test for EmptyStr


val r3 = EmptyStr
testWithMessage(delta(r3, 'a'), Null, "#3")

// Test for Null

val r4 = Null
testWithMessage(delta(r4, 'a'), Null, "#4")

// Atom empty str
val r5 = Atom("")
testWithMessage(delta(r5, 'x'), Null, "#5")

passed(5)

Test #1
	 Your code returned: Atom(ello), Expected: Atom(ello)
	 Passed!
Test #2
	 Your code returned: Null, Expected: Null
	 Passed!
Test #3
	 Your code returned: Null, Expected: Null
	 Passed!
Test #4
	 Your code returned: Null, Expected: Null
	 Passed!
Test #5
	 Your code returned: Null, Expected: Null
	 Passed!

*** Tests Passed (5 points) ***


r1: Atom = Atom(s = "hello")
e1: Atom = Atom(s = "ello")
r2: Atom = Atom(s = "hello")
e2: Null.type = Null
r3: EmptyStr.type = EmptyStr
r4: Null.type = Null
r5: Atom = Atom(s = "")

In [11]:
// Test Or only
val r7 = "hello" | "world"
val e7 = Null | "orld"
testWithMessage(delta(r7,'w'), e7, "#6")
// Test Or and star
val r6 = "hello" | "world".star 
val e6 = "ello" | (Null o "world".star)
testWithMessage(delta(r6,'h'), e6, "#7")
// Test Star
val r8 = "star".star 
val e8 = "tar" o r8
testWithMessage(delta(r8, 's'), e8, "#8")
// Test Star
val r9 = "".star
testWithMessage(delta(r9, 'x'), Null o r9, "#9")
passed(5)

Test #6
	 Your code returned: Or(Null,Atom(orld)), Expected: Or(Null,Atom(orld))
	 Passed!
Test #7
	 Your code returned: Or(Atom(ello),Seq(Null,Star(Atom(world)))), Expected: Or(Atom(ello),Seq(Null,Star(Atom(world))))
	 Passed!
Test #8
	 Your code returned: Seq(Atom(tar),Star(Atom(star))), Expected: Seq(Atom(tar),Star(Atom(star)))
	 Passed!
Test #9
	 Your code returned: Seq(Null,Star(Atom())), Expected: Seq(Null,Star(Atom()))
	 Passed!

*** Tests Passed (5 points) ***


r7: Or = Or(r1 = Atom(s = "hello"), r2 = Atom(s = "world"))
e7: Or = Or(r1 = Null, r2 = Atom(s = "orld"))
r6: Or = Or(r1 = Atom(s = "hello"), r2 = Star(r = Atom(s = "world")))
e6: Or = Or(
  r1 = Atom(s = "ello"),
  r2 = Seq(r1 = Null, r2 = Star(r = Atom(s = "world")))
)
r8: Star = Star(r = Atom(s = "star"))
e8: Seq = Seq(r1 = Atom(s = "tar"), r2 = Star(r = Atom(s = "star")))
r9: Star = Star(r = Atom(s = ""))

In [12]:
// Test Seq
val r10 = "hello" o "world"
val e10: Regex = "ello" o "world"
testWithMessage(delta(r10, 'h'), e10, "#10")
testWithMessage(delta(r10, 'k'), Null o "world", "#11")

val r11 = "" o "world"
val e11 = (Null o "world") | "orld"
testWithMessage(delta(r11, 'w'), e11, "#12")

val r12 = Star("world") o "water"
val e12 = (("orld" o Star("world")) o "water") | "ater"
testWithMessage(delta(r12,'w'), e12, "#13")
passed(5)


Test #10
	 Your code returned: Seq(Atom(ello),Atom(world)), Expected: Seq(Atom(ello),Atom(world))
	 Passed!
Test #11
	 Your code returned: Seq(Null,Atom(world)), Expected: Seq(Null,Atom(world))
	 Passed!
Test #12
	 Your code returned: Or(Seq(Null,Atom(world)),Atom(orld)), Expected: Or(Seq(Null,Atom(world)),Atom(orld))
	 Passed!
Test #13
	 Your code returned: Or(Seq(Seq(Atom(orld),Star(Atom(world))),Atom(water)),Atom(ater)), Expected: Or(Seq(Seq(Atom(orld),Star(Atom(world))),Atom(water)),Atom(ater))
	 Passed!

*** Tests Passed (5 points) ***


r10: Seq = Seq(r1 = Atom(s = "hello"), r2 = Atom(s = "world"))
e10: Regex = Seq(r1 = Atom(s = "ello"), r2 = Atom(s = "world"))
r11: Seq = Seq(r1 = Atom(s = ""), r2 = Atom(s = "world"))
e11: Or = Or(r1 = Seq(r1 = Null, r2 = Atom(s = "world")), r2 = Atom(s = "orld"))
r12: Seq = Seq(r1 = Star(r = Atom(s = "world")), r2 = Atom(s = "water"))
e12: Or = Or(
  r1 = Seq(
    r1 = Seq(r1 = Atom(s = "orld"), r2 = Star(r = Atom(s = "world"))),
    r2 = Atom(s = "water")
  ),
  r2 = Atom(s = "ater")
)

We are now ready to implement our regular expression matcher. You are given  a regex `r` and a string `s`. We wish to know if the entire string `s` matches `r`. We do the following: 

  1. Start with initial regex `r`
  2. Iterate through each character `c` of the string:
     - compute derivative $\delta(r, c)$, call the result $r'$.
     - assign $r'' = \sNull(r')$ and update $r$ to equal $r''$.
  3. If at the end of the string, $\acc(r)$ is true then we have a match, else we do not have a match.

Here is a loopy version for your reference.

In [13]:
def match_loop(r0: Regex, s: String) : Boolean = {
    var r = r0
    for (c <- s) { // Iterate through the string
        r  = simplifyNull(delta(r, c))
    }
    accepts(r)
}

defined function match_loop

In [14]:
val r12 = Star("world") o "water"
println("1 -> "+ match_loop(r12, "worldworldwater"))
println("2 -> "+match_loop(r12, "worldwaterwater"))
println("3 -> "+match_loop(r12, "water"))
println("4 -> "+match_loop(r12, "waterworldworld"))

val r13 = ("x" o ("_yellow").star).star | ("_white")
println("5 -> "+match_loop(r13, "x"))
println("6 -> "+match_loop(r13, "x_yellow_yellowx_yellow_yellow_yellowxxxxx"))
println("7 -> "+match_loop(r13, "_white"))

val r14 = r13.star
println("8 ->" +  match_loop(r14,"x_yellow_whitex_yellow_yellow_whitexxxx_yellow_whitex"))


1 -> true
2 -> false
3 -> true
4 -> false
5 -> true
6 -> true
7 -> true
8 ->true


r12: Seq = Seq(r1 = Star(r = Atom(s = "world")), r2 = Atom(s = "water"))
r13: Or = Or(
  r1 = Star(r = Seq(r1 = Atom(s = "x"), r2 = Star(r = Atom(s = "_yellow")))),
  r2 = Atom(s = "_white")
)
r14: Star = Star(
  r = Or(
    r1 = Star(r = Seq(r1 = Atom(s = "x"), r2 = Star(r = Atom(s = "_yellow")))),
    r2 = Atom(s = "_white")
  )
)

Just like lists, scala String API has fold operation (see example below). Use fold left instead of loop to match a string to a regex.

In [15]:
val str = "Hello"

val count_num_l = str.foldLeft[Int] (0) { 
    case (acc, char) => if (char == 'l') { acc + 1 } else { acc }
}



str: String = "Hello"
count_num_l: Int = 2

In [16]:
/* Restriction: Code has to use foldLeft on string -- 
   no vars and loops allowed */

def fold_match(r: Regex, s: String) : Boolean = {
   val store = s.foldLeft(r)
    {
        case (e1, c) => simplifyNull(delta(e1, c))
    }
     accepts(store)
}


defined function fold_match

In [17]:
val r12 = Star("world") o "water"
testWithMessage( fold_match(r12, "worldworldwater"), match_loop(r12, "worldworldwater"), "#1")
testWithMessage( fold_match(r12, "worldwaterwater"), match_loop(r12, "worldwaterwater"), "#2")
testWithMessage( fold_match(r12, "water"), match_loop(r12, "water"), "#3")
testWithMessage(fold_match(r12, "waterworldworld"), match_loop(r12, "waterworldworld"), "#4")

val r13 = ("x" o ("_yellow").star).star | ("_white")
testWithMessage(fold_match(r13,"x"), match_loop(r13, "x"), "#5")
testWithMessage(fold_match(r13, "x_yellow_yellowx_yellow_yellow_yellowxxxxx"),
                match_loop(r13, "x_yellow_yellowx_yellow_yellow_yellowxxxxx"),
                "#6")
testWithMessage(fold_match(r13, "_white"), match_loop(r13, "_white"), "#7")

val r14 = r13.star
testWithMessage( fold_match(r14,"x_yellow_whitex_yellow_yellow_whitexxxx_yellow_whitex"),
                match_loop(r14,"x_yellow_whitex_yellow_yellow_whitexxxx_yellow_whitex"), "#8")

passed(10)

Test #1
	 Your code returned: true, Expected: true
	 Passed!
Test #2
	 Your code returned: false, Expected: false
	 Passed!
Test #3
	 Your code returned: true, Expected: true
	 Passed!
Test #4
	 Your code returned: false, Expected: false
	 Passed!
Test #5
	 Your code returned: true, Expected: true
	 Passed!
Test #6
	 Your code returned: true, Expected: true
	 Passed!
Test #7
	 Your code returned: true, Expected: true
	 Passed!
Test #8
	 Your code returned: true, Expected: true
	 Passed!

*** Tests Passed (10 points) ***


r12: Seq = Seq(r1 = Star(r = Atom(s = "world")), r2 = Atom(s = "water"))
r13: Or = Or(
  r1 = Star(r = Seq(r1 = Atom(s = "x"), r2 = Star(r = Atom(s = "_yellow")))),
  r2 = Atom(s = "_white")
)
r14: Star = Star(
  r = Or(
    r1 = Star(r = Seq(r1 = Atom(s = "x"), r2 = Star(r = Atom(s = "_yellow")))),
    r2 = Atom(s = "_white")
  )
)

Congratulations! You have now implemented a fairly decent regex matcher using functional programming principles. We will eliminate the use of recursion later in this course using continuations and trampolines. 

## Problem 2: map, filter, reduce on containers (20 points)

Solve the problems using a combination of map, filter and foldLeft/foldRight opertions over lists. The use of mutables, recursion, For/While loops is forbidden for this problem.


### 2A (5 points): Compute the dot-product of two lists of numbers.
Write a function `computeDotProduct` that takes two lists `lstA` and `lstB` of double precision numbers. The lists are given to be the same size. You need to compute the dot product. 

$$(a_0, \ldots, a_n) \cdot (b_0, \ldots, b_n) = \sum_{j=0}^n a_j b_j $$.

List API functions that you are allowed to use: `zip`,  `map`, `filter`, `foldLeft`, and `sum`. You can look the list API documentation for scala to find out more about these functions. Post/Search on piazza if you are unsure. You are __not__ allowed to use __var__, any kind of loops or recursion.

In [18]:
def dotProduct(lstA: List[Double], lstB: List[Double]): Double =
    
    {
        require(lstA.length == lstB.length)
        // YOUR CODE HERE
        val lst1 = lstA.zip(lstB)
            .map{
                case num:(Double,Double) => {num._1 * num._2}
            }
            .sum
        
        return lst1
    }
// val a = dotProduct(List(1.1,2.0), List(3.0, 4.2))
// a

defined function dotProduct

In [19]:
// BEGIN TEST

val t1 = dotProduct(List(1.1,2.0), List(3.0, 4.2))
assert(math.abs(t1 - 11.7)<= 1E-08, "Test 1 failed")

val t2 = dotProduct(List(1.1), List(2.0))
assert(math.abs(t2 - 2.2)<= 1E-08, "Test 2 failed")

val t3 = dotProduct(List(), List())
assert(math.abs(t3)<= 1E-08, "Test 3 failed")

val t4 = dotProduct(List(1.5, 0.0, 2.3, -1.1, 0.0), List(0.0, 4.5, 1.1, 2.3, 5.0))
assert(math.abs(t4) <= 1E-08, "Test 4 failed")

passed(5)
// END TEST


*** Tests Passed (5 points) ***


t1: Double = 11.700000000000001
t2: Double = 2.2
t3: Double = 0.0
t4: Double = 0.0

### 2B (5 points): LCM of all Denominators.

You are given a list of pairs of integers that are supposed to represent fractions. You can assume that all fractions are positive and already in their lowest terms. This problem asks you to compute the LCM of the denominators.

Eg., Input: `List((1,2), (3,4), (8,9), (15,24))`
The denominators are `2, 4, 9, 24` and their LCM is `72`, which is the answer your function should return.

For the empty list as input, your code must return the answer 1.

You should assume that all the denominators are positive.

You are allowed to use the provided `lcm` function that computes LCM of two numbers, `map`, `filter`, `foldLeft`, and `foldRight`. No __var__, no loops and no recursive calls other than the call made for LCM.


In [20]:
import scala.annotation._

@tailrec
private def gcd(a: Int, b: Int):Int=if (b==0) a.abs else gcd(b, a%b)

def lcm(a: Int, b: Int)=(a*b).abs/gcd(a,b)

def lcmOfDenominators(lst: List[(Int, Int)]): Int = {
    // YOUR CODE HERE
    val lst1 = lst.map {
        case num:(Int, Int) =>(num._2)
    }
    lst1.foldLeft(1)(lcm)
}

import scala.annotation._


defined function lcm
defined function lcmOfDenominators

In [21]:
// BEGIN TEST
val i1 = lcmOfDenominators(List((1,2), (3,4), (5,6),(8,9), (11,12)))
assert(i1 == 36, s"Test 1 failed -- expected answer is 36, you got $i1")
val i2 = lcmOfDenominators(List())
assert(i2 == 1, s"Test 2 failed -- empty list must trivially have lcm of 1")
val i3 = lcmOfDenominators(List((4,5),(5,9),(18,15)))
assert(i3 == 45, s"Test 2 failed -- your answer is $i3")
passed(5)
// END TEST


*** Tests Passed (5 points) ***


i1: Int = 36
i2: Int = 1
i3: Int = 45

### 2C (10 points): Convert a list into an indexed list.

Write a function to convert a list of strings into an indexed list of strings.  Indices start at 0.
As an example:

__Input__ List("hello", "world", "my", "cat", "is", "grumpy", "today")

__Output__  List( (0, "hello"), (1, "world"), (2, "my"), (3,"cat"), (4, "is"), (5, "grumpy"), (6,"today") )


You are allowed to use just the basic list operatios such as cons of an element to a list (::) and concatenation of two lists (++, or ::: operators). List API functions `reverse`, `map`, `filter`, `foldLeft` and `foldRight` but not other list API functions. Do not use __var__, __loops__ or __recursion__. Use of `zipWithIndex` or other API functions  is forbidden.

In [22]:
def makeIndexedList(lst:List[String]): List[(Int, String)] = {
    // YOUR CODE HERE

    lst.foldLeft(0, List[(Int, String)]())
    {
       case ((num, nList), s)  => (num+1, nList :+(num, s))
    }._2
    
    
}

defined function makeIndexedList

In [23]:
// BEGIN TEST
val t1 = makeIndexedList(List("hello"))
assert(t1 == List((0,"hello")), s"Test 1 failed - your code returned $t1")

val t2 = makeIndexedList(List("hello", "world"))
assert(t2 == List((0,"hello"), (1, "world")), s"Test 2 failed - your code returned $t2")

val t3 = makeIndexedList(Nil)
assert(t3 == Nil, s"Test 3 failed - your code returned $t3")

val t4 = makeIndexedList(List("a","b","c","d","e"))
assert(t4 == List((0,"a"), (1,"b"), (2,"c"), (3,"d"), (4,"e")), s"Test 4 failed - your code returned $t4")

passed(10)
// END TEST


*** Tests Passed (10 points) ***


t1: List[(Int, String)] = List((0, "hello"))
t2: List[(Int, String)] = List((0, "hello"), (1, "world"))
t3: List[(Int, String)] = List()
t4: List[(Int, String)] = List((0, "a"), (1, "b"), (2, "c"), (3, "d"), (4, "e"))

## That's All Folks!